# Homework 4, Bill Wang Hong

## Part 2. Runner Notebook

In [1]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import sqlite3
import pandas as pd
import os
import numpy as np
import gspread
import scrapbook as sb

import os

Firstly, import the data from the sql database.

Purpose of this is to understand the tokens and dates included in the database

In [2]:
prices = pd.read_sql(f'''
SELECT 
    * 
FROM ohlc 
''', 'sqlite:///data/data.db')

data_all = pd.DataFrame(
    prices,
    columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD', 'token', 'chain']
)

data_all['ts'] = pd.to_datetime(data_all['ts'])

Then, find out about the list of tokens and the last 2 complete weeks

In [3]:
#Find the list of tokens
tokens = set(data_all["token"])

#Find complete last day
last_day = max(data_all['ts'])
if last_day.hour < 23:
    last_complete_day = datetime(last_day.year, last_day.month,last_day.day-1)
else:
    last_complete_day = last_day

#Find the start date of the last 2 complete weeks (Monday to Sunday)
first_day = last_complete_day - timedelta(days=14+last_complete_day.weekday())
run_dates = [first_day + timedelta(days=7*i) for i in range(2)]

connection_string = 'sqlite:///data/data.db'
events_db = 'crypto_events_database'

Next we run the pre-defined template

In [4]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [5]:
#            Please note that i did not use the following naming convention of the output as my anaconda does not allow space inside naming convention
#            f'{base_output_folder}/hw4_{token}__{run_date}.ipynb',


for run_date in run_dates:
    for token in tokens:
        print(f'running for {token} for date {run_date} to {run_date + timedelta(days=7)}', end='\r')
        res = pm.execute_notebook(
            'hw4_hong_wang_template.ipynb',
            f'{base_output_folder}/hw4_{token}_{str(run_date.year) + "-" + str(run_date.month) + "-" +  str(run_date.day)}.ipynb',
            parameters = dict(from_date=f'{run_date}', to_date = f'{run_date + timedelta(days=7)}', 
                             token = token, connection_string = connection_string))

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

Executing:   0%|          | 0/18 [00:00<?, ?cell/s]

#### Finally, compute the following: 
- avg hourly return by run
- avg volatility by run
- maximum drawdown over the entire period of each run
- range of last close price - first close price

In [6]:
nbs = sb.read_notebooks('outputs')

In [7]:
pd.DataFrame([
    pd.Series({
        'run_name': book,
        'token': scrap['token'].data,
        'from_date': scrap['from_date'].data,
        'to_date': scrap['to_date'].data,
        'avg_hourly_return': scrap['data_DF'].data.volatility.mean(),
        'volatility': scrap['data_DF'].data.volatility.mean(),
        'max_drawdown': scrap['data_DF'].data.max_draw.min(),
        'last_minus_first_close_price': scrap['data_DF'].data.close[167] - scrap['data_DF'].data.close[0]
    }) 
    for book, scrap in nbs.notebook_scraps.items()])

,run_name,token,from_date,to_date,avg_hourly_return,volatility,max_drawdown,last_minus_first_close_price
0,hw4_AAVE_2021-11-29,AAVE,2021-11-29 00:00:00,2021-12-06 00:00:00,0.221054,0.221054,-0.302593,-46.8080
1,hw4_AAVE_2021-12-6,AAVE,2021-12-06 00:00:00,2021-12-13 00:00:00,0.200400,0.200400,-0.179887,-11.6730
2,hw4_ADA_2021-11-29,ADA,2021-11-29 00:00:00,2021-12-06 00:00:00,0.226749,0.226749,-0.250000,-0.2305
3,hw4_ADA_2021-12-6,ADA,2021-12-06 00:00:00,2021-12-13 00:00:00,0.203766,0.203766,-0.177991,-0.0297
4,hw4_ATOM_2021-11-29,ATOM,2021-11-29 00:00:00,2021-12-06 00:00:00,0.340796,0.340796,-0.265123,-2.8370
5,hw4_ATOM_2021-12-6,ATOM,2021-12-06 00:00:00,2021-12-13 00:00:00,0.254383,0.254383,-0.131744,-0.0040
6,hw4_AVAX_2021-11-29,AVAX,2021-11-29 00:00:00,2021-12-06 00:00:00,0.320661,0.320661,-0.334566,-25.6000
7,hw4_AVAX_2021-12-6,AVAX,2021-12-06 00:00:00,2021-12-13 00:00:00,0.249173,0.249173,-0.165626,1.1800
8,hw4_BTC_2021-11-29,BTC,2021-11-29 00:00:00,2021-12-06 00:00:00,0.175543,0.175543,-0.206462,-7979.4100
9,hw4_BTC_2021-12-6,BTC,2021-12-06 00:00:00,2021-12-13 00:00:00,0.134449,0.134449,-0.089371,833.1100
